In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import libraries

In [ ]:

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# from sklearn.externals import joblib
import joblib
from sklearn.pipeline import Pipeline
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

### Load data

In [ ]:
true_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')
fake_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
print(f'shape True New articles: {true_df.shape}')
print(f'shape True New articles: {true_df.shape}')

In [ ]:
true_df['Label']='True'
fake_df['Label']='Fake'

### clean and preprocess the 2 datasets and combine them

In [ ]:
df_final=pd.concat([true_df,fake_df])
print(f'shape final data set: {df_final.shape}')

### Preprocess and spilit data set into training and test using pipline methond
* also add ML model to classify the fake vs true articles

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df_final["text"], df_final["Label"],random_state = 10,test_size = 0.20)
print(f'Train feature set: {X_train.shape}')
print(f'Train Lable set: {y_train.shape}')

In [ ]:
# Building Pipeline for raw text transformation
clf = Pipeline([
    ('vect', CountVectorizer(stop_words= "english")),
   # ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
    ])

In [ ]:
model_nb= clf.fit(X_train,y_train)

In [ ]:
score=model_nb.score(X_test,y_test)
print(f'Accuracy of model :{score}')

In [ ]:
y_predicted = model_nb.predict(X_test)
y_predicted[0:10]

### validate the score
* Confusion matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_predicted)
np.set_printoptions(precision=2)
cnf_matrix

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure(figsize= (8,8))
plot_confusion_matrix(cnf_matrix, classes= np.sort(df_final["Label"].unique()),
                      title='Confusion matrix, without normalization')
# With normalization
plt.figure(figsize= (8,8))
plot_confusion_matrix(cnf_matrix, classes= np.sort(df_final["Label"].unique())
                      , normalize=True,title='Normalized confusion matrix')

plt.show()

### save the trained model in file system

In [ ]:
joblib.dump(model_nb,'navies.joblib')

In [ ]:
model_load=joblib.load('navies.joblib')

In [ ]:
sample="You have won %5000000000 lottery"

In [ ]:
model_load.predict([sample])